In [ ]:
import tensorflow as tf
import numpy as np

import sionna as sn

%matplotlib inline
# also try %matplotlib widget

import matplotlib.pyplot as plt

import time

from tensorflow.keras import Model

In [ ]:
import math
from pathlib import Path

from example import (
    save_image_raw,
    load_image_raw,
    unpack_bytes_to_bits,
    pack_bits_to_bytes,
    get_psnr,
)


class CodedSystemAWGN(Model):  # Inherits from Keras Model
    def __init__(
        self,
        image_file,
        reference_file,
        num_bits_per_symbol,
        block_len,
        coderate,
        do_print: bool = True,
    ):
        super().__init__()  # Must call the Keras model initializer

        self.do_print = do_print
        self.image_file = image_file
        self.reference_img_bytes, _ = load_image_raw(reference_file)
        self.num_bits_per_symbol = num_bits_per_symbol
        self.block_len = block_len
        self.k = int(block_len * coderate)
        self.coderate = coderate
        self.constellation = sn.mapping.Constellation("qam", self.num_bits_per_symbol)

        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation)

        self.binary_source = sn.utils.BinarySource()
        self.awgn_channel = sn.channel.AWGN()

        self.encoder = sn.fec.ldpc.LDPC5GEncoder(self.k, self.block_len)
        self.decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=True)
        # self.encoder = sn.fec.polar.Polar5GEncoder(self.k, self.block_len)
        # self.decoder = sn.fec.polar.Polar5GDecoder(self.encoder, dec_type="SCL")

    # @tf.function # activate graph execution to speed things up
    def __call__(self, batch_size, ebno_db, save: Path | None):
        no = sn.utils.ebnodb2no(
            ebno_db,
            num_bits_per_symbol=self.num_bits_per_symbol,
            coderate=self.coderate,
        )

        inp_img_bytes, _ = load_image_raw(self.image_file)
        inp_bits = unpack_bytes_to_bits(inp_img_bytes, 8).reshape(-1)

        bits_len = len(inp_bits)
        size = math.ceil(bits_len / batch_size / self.k) * batch_size * self.k
        inp_bits = np.pad(inp_bits, (0, size - bits_len))
        out_bits = np.zeros_like(inp_bits)

        group_size = self.k * batch_size

        for i in range(int(size / group_size)):
            if self.do_print:
                print(f"{i * group_size}...{(i+1)*group_size} / {size}")
            inp = inp_bits[(i * group_size) : ((i + 1) * group_size)].reshape(
                (batch_size, self.k)
            )

            bits = tf.convert_to_tensor(inp, dtype=float)
            # bits = self.binary_source([batch_size, self.k])
            codewords = self.encoder(bits)
            x = self.mapper(codewords)
            y = self.awgn_channel([x, no])
            llr = self.demapper([y, no])
            bits_hat = self.decoder(llr)

            out_bits[
                (i * group_size) : ((i + 1) * group_size)
            ] = bits_hat.numpy().reshape(batch_size * self.k)

        # only the first group
        # inp_bits = inp_bits[: (self.k * batch_size)]
        # inp_bits = inp_bits.reshape((batch_size, self.k))

        # bits = tf.convert_to_tensor(inp_bits, dtype=float)
        # # bits = self.binary_source([batch_size, self.k])
        # codewords = self.encoder(bits)
        # x = self.mapper(codewords)
        # y = self.awgn_channel([x, no])
        # llr = self.demapper([y, no])
        # bits_hat = self.decoder(llr)

        # truncate transmitted bits back to input size
        inp_bits = inp_bits[:bits_len]
        out_bits = out_bits[:bits_len]

        ber = float(sn.utils.BitErrorRate()(inp_bits, out_bits))
        out_img_bytes = pack_bits_to_bytes(out_bits)
        psnr = get_psnr(self.reference_img_bytes, out_img_bytes)
        if save is not None:
            save_image_raw(
                save
                , out_img_bytes
            )

        return inp_bits, out_bits, ber, psnr

In [ ]:
import itertools

BATCH_SIZE = 64  # How many examples are processed by Sionna in parallel
BLOCKLENGTH = 1024
CODERATE = 0.5


num_bits_per_symbol = [2, 4, 8]  # 2 = QPSK
ebn0s = [1, 1.5, 2, 2.5, 3, 4, 4.5, 5, 10, 10.5, 11, 11.5, 12]

bers = {nbits: [] for nbits in num_bits_per_symbol}
psnrs = {nbits: [] for nbits in num_bits_per_symbol}

for nbits, ebn0 in itertools.product(num_bits_per_symbol, ebn0s):
    model_coded_awgn = CodedSystemAWGN(
        "kodim23.jpeg",
        "kodim23.png",
        # "../experiments/test_data/test/orig_ref_002.jpg",
        # "../experiments/test_data/test/orig_ref_002.png",
        num_bits_per_symbol=nbits,
        block_len=BLOCKLENGTH,
        coderate=CODERATE,
        do_print=True,
    )

    out_img = f"../experiments/digcom/out_{nbits}bps_EbN0_{ebn0}dB.jpg"

    bits, bits_hat, ber, psnr = model_coded_awgn(BATCH_SIZE, ebn0, save=out_img)
    print(f"nbits {nbits:2d}, Eb/N0 {ebn0:.2f}dB: BER {ber:10.8f}, PSNR {psnr:7.3f} dB")

    bers[nbits].append(ber)
    psnrs[nbits].append(psnr)

plt.figure()
for nbits in num_bits_per_symbol:
    plt.plot(ebn0s, bers[nbits], label=f"{nbits} bits")
plt.yscale("log")
plt.title("BER")
plt.legend()
plt.xlabel("Eb/N0 (dB)")
plt.ylabel("BER (-)")
# plt.savefig("ber.png", dpi=300)

# plt.clf()

plt.figure()
for nbits in num_bits_per_symbol:
    plt.plot(ebn0s, psnrs[nbits], label=f"{nbits} bits")
plt.title("PSNR")
plt.legend()
plt.xlabel("Eb/N0 (dB)")
plt.ylabel("PSNR (dB)")
# plt.savefig("psnr.png", dpi=300)

plt.show()

In [ ]:
NUM_BITS_PER_SYMBOL = 2  # QPSK
constellation = sn.mapping.Constellation("qam", NUM_BITS_PER_SYMBOL)

# constellation.show()

In [ ]:
mapper = sn.mapping.Mapper(constellation=constellation)

# The demapper uses the same constellation object as the mapper
demapper = sn.mapping.Demapper("app", constellation=constellation)

binary_source = sn.utils.BinarySource()
awgn_channel = sn.channel.AWGN()

no = sn.utils.ebnodb2no(
    ebno_db=10.0, num_bits_per_symbol=NUM_BITS_PER_SYMBOL, coderate=1.0
)  # Coderate set to 1 as we do uncoded transmission here


In [ ]:
import math
from example import load_image_raw, unpack_bytes_to_bits

BATCH_SIZE = 64  # How many examples are processed by Sionna in parallel
BLOCKLENGTH = 1024

inp_img, resolution = load_image_raw("kodim23.jpeg")
inp_bits = unpack_bytes_to_bits(inp_img, 8).reshape(-1)

bits_len = len(inp_bits)
size = math.ceil(bits_len / BATCH_SIZE / BLOCKLENGTH) * BATCH_SIZE * BLOCKLENGTH
inp_bits = np.pad(inp_bits, (0, size - bits_len))

# only first group
inp_bits = inp_bits[: (BLOCKLENGTH * BATCH_SIZE)]

inp_bits = inp_bits.reshape((BATCH_SIZE, BLOCKLENGTH))
bits = tf.convert_to_tensor(inp_bits, dtype=float)
# bits = binary_source([BATCH_SIZE, BLOCKLENGTH])  # Blocklength
print("Shape of bits: ", bits.shape)
print("Type of bits: ", type(bits), bits.dtype, bits.dtype.as_numpy_dtype)

x = mapper(bits)
print("Shape of x: ", x.shape)

y = awgn_channel([x, no])
print("Shape of y: ", y.shape)

llr = demapper([y, no])
print("Shape of llr: ", llr.shape)
